In [20]:
from torchsummaryX import summary
import torch
from package.config import Config
from model.speller import Speller
from model.listener import Listener
from model.listenAttendSpell import ListenAttendSpell

config = Config(
    use_bidirectional=True,
    use_label_smooth=True,
    input_reverse=True,
    use_augment=False,
    use_pickle=False,
    use_cuda=True,
    augment_ratio=1.0,
    hidden_dim=256,
    dropout=0.5,
    listener_layer_size=5,
    speller_layer_size=3,
    batch_size=12,
    worker_num=1,
    max_epochs=40,
    use_multistep_lr=False,
    init_lr=0.0001,
    high_plateau_lr=0.0003,
    low_plateau_lr=0.00001,
    teacher_forcing=1.0,
    seed=1,
    max_len=151,
    load_model=False,
    model_path=None
)

listener = Listener(
    in_features=80,
    hidden_dim=config.hidden_dim,
    dropout_p=config.dropout,
    n_layers=config.listener_layer_size,
    bidirectional=config.use_bidirectional,
    rnn_type='gru',
    device='cpu'
)
speller = Speller(
    n_class=2040,
    max_length=config.max_len,
    k=8,
    hidden_dim=config.hidden_dim << (1 if config.use_bidirectional else 0),
    sos_id=2037,
    eos_id=2038,
    n_layers=config.speller_layer_size,
    rnn_type='gru',
    dropout_p=config.dropout,
    device='cpu'
)
model = ListenAttendSpell(listener, speller)

batch_size = 32
seq_length = 120
feature_size = 80
max_length = 150

inputs = torch.zeros((batch_size, seq_length, feature_size))
scripts = torch.zeros((batch_size, seq_length, max_length), dtype=torch.long)
summary(model, inputs, scripts)

[2020-04-27 02:37:17,709 config.py:91 - print_log()] use_bidirectional : True
[2020-04-27 02:37:17,712 config.py:92 - print_log()] use_pickle : False
[2020-04-27 02:37:17,715 config.py:93 - print_log()] use_augment : False
[2020-04-27 02:37:17,718 config.py:94 - print_log()] augment_ratio : 1.00
[2020-04-27 02:37:17,721 config.py:95 - print_log()] input_reverse : True
[2020-04-27 02:37:17,725 config.py:96 - print_log()] hidden_dim : 256
[2020-04-27 02:37:17,727 config.py:97 - print_log()] listener_layer_size : 5
[2020-04-27 02:37:17,730 config.py:98 - print_log()] speller_layer_size : 3
[2020-04-27 02:37:17,732 config.py:99 - print_log()] dropout : 0.50
[2020-04-27 02:37:17,735 config.py:100 - print_log()] batch_size : 12
[2020-04-27 02:37:17,739 config.py:101 - print_log()] worker_num : 1
[2020-04-27 02:37:17,741 config.py:102 - print_log()] max_epochs : 40
[2020-04-27 02:37:17,744 config.py:103 - print_log()] initial learning rate : 0.0001
[2020-04-27 02:37:17,746 config.py:107 - pri

,Kernel Shape,Output Shape,Params,Mult-Adds
Layer,,,,
0_listener.conv.Conv2d_0,"[1, 64, 3, 3]","[32, 64, 120, 80]",640.0,5529600.0
1_listener.conv.Hardtanh_1,-,"[32, 64, 120, 80]",NaN,NaN
2_listener.conv.BatchNorm2d_2,[64],"[32, 64, 120, 80]",128.0,64.0
3_listener.conv.Conv2d_3,"[64, 64, 3, 3]","[32, 64, 120, 80]",36928.0,353894400.0
4_listener.conv.Hardtanh_4,-,"[32, 64, 120, 80]",NaN,NaN
5_listener.conv.MaxPool2d_5,-,"[32, 64, 60, 40]",NaN,NaN
6_listener.conv.BatchNorm2d_6,[64],"[32, 64, 60, 40]",128.0,64.0
7_listener.conv.Conv2d_7,"[64, 128, 3, 3]","[32, 128, 60, 40]",73856.0,176947200.0
8_listener.conv.Hardtanh_8,-,"[32, 128, 60, 40]",NaN,NaN
